**今回の実習内容**
 * 逆空間を利用した画像の回転
 * メタデータの読み込み
 * 逆投影
 * 実習課題

In [ ]:
import sys,math,cmath,os,copy
import numpy as np
from numpy import fft
from scipy import ndimage
from matplotlib import pyplot as plt
import tifffile as tiff
import mrcfile as mrc

**自分で作ったモジュールのインポート**
 * モジュールファイルは.py形式で中に関数やクラスを定義しておく
 * インポートすればnumpyやtifffileのようにモジュール内の関数を利用できる
 * モジュールがあるディレクトリをsys.pathに追加するとインポート可能になる

In [ ]:
# srcの中にあるモジュール"projector.py", "backprojector.py"をインポート
sys.path.append('./src/')
import projector
import backprojector

**逆空間を利用した画像の回転**

In [ ]:
# テストデータの読み込み
fn_bgal = "./data/course5_bgal.tif"
bgal = tiff.imread(fn_bgal)
print("image size of bgal: ", bgal.shape[0], " x ", bgal.shape[1])

# 2xパディングした画像を作る
factor = 2
margin = int(bgal.shape[0] * (factor - 1) / 2)
bgal_pad = np.pad(bgal, (margin,margin))

print("image size of bgal_pad: ", bgal_pad.shape[0], " x ", bgal_pad.shape[1])

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(8,4))

ax[0].imshow(bgal, cmap='Greys')
ax[1].imshow(bgal_pad, cmap='Greys')

In [ ]:
# テストデータを20度回転してみる
psi = 20

A = np.zeros((3,3))
projector.getRotationMatrix(A, rot=0, tilt=0, psi=psi)

# 逆空間に持っていく
rec_bgal = fft.fftshift(fft.fft2(fft.fftshift(bgal)))
rec_bgal_pad = fft.fftshift(fft.fft2(fft.fftshift(bgal_pad)))

# 逆空間で回転させる
rec_bgal_rot = np.zeros(rec_bgal.shape, dtype=complex)
rec_bgal_pad_rot = np.zeros(rec_bgal_pad.shape, dtype=complex)

projector.rotate2D(rec_bgal, rec_bgal_rot, A)
projector.rotate2D(rec_bgal_pad, rec_bgal_pad_rot, A)

# 実部のみを取り出し
bgal_rot = fft.ifftshift(fft.ifft2(fft.ifftshift(rec_bgal_rot))).real
bgal_pad_rot = fft.ifftshift(fft.ifft2(fft.ifftshift(rec_bgal_pad_rot))).real

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(8,4))

ax[0].imshow(bgal_rot, cmap='Greys')
ax[1].imshow(bgal_pad_rot[margin:bgal.shape[0]+margin,margin:bgal.shape[1]+margin], cmap='Greys') # パディングしたところをトリムして表示


In [ ]:
# 20度回転を9回繰り返して180度回転させた時の違いを見る

# 回転行列は使い回すので先に計算しておく
psi = 20
A = np.zeros((3,3))
projector.getRotationMatrix(A, rot=0, tilt=0, psi=psi)

bgal_store = copy.deepcopy(bgal)
bgal_pad_store = copy.deepcopy(bgal_pad)

ntrial = 9
for n in range(ntrial):
    print('current rotated angle: ', (n+1)*psi)
    rec_bgal = fft.fftshift(fft.fft2(fft.fftshift(bgal_store)))
    rec_bgal_pad = fft.fftshift(fft.fft2(fft.fftshift(bgal_pad_store)))
    
    rec_bgal_rot = np.zeros(rec_bgal.shape, dtype=complex)
    rec_bgal_pad_rot = np.zeros(rec_bgal_pad.shape, dtype=complex)

    projector.rotate2D(rec_bgal, rec_bgal_rot, A)
    projector.rotate2D(rec_bgal_pad, rec_bgal_pad_rot, A)

    # 実部のみを取り出し
    bgal_store = fft.ifftshift(fft.ifft2(fft.ifftshift(rec_bgal_rot))).real
    bgal_pad_store = fft.ifftshift(fft.ifft2(fft.ifftshift(rec_bgal_pad_rot))).real

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(8,4))

ax[0].imshow(bgal_store, cmap='Greys')
ax[1].imshow(bgal_pad_store[margin:bgal.shape[0]+margin,margin:bgal.shape[1]+margin], cmap='Greys') # パディングしたところをトリムして表示


**画像とメタデータの読み込み**
 * 再構成をするためには投影像だけでなく、各投影像の配向が必要
 * 投影像の名前と配向をセットにしたメタデータファイルを読み込む
 * 数値だけのデータの場合シンプルな読み込みが可能
 * pandasというモジュールを使うことで色々なことができるが、ここでは説明しない

In [ ]:
# 投影像を読み込み
fn_stack = "./data/course6_bgal_proj.mrcs"
stack = mrc.read(fn_stack)
print("Number of particles:", stack.shape[0])
print("Particle image size: ", stack.shape[1], " x ", stack.shape[2])

fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(12,3))

ax[0].imshow(stack[0,:,:], cmap='Greys')
ax[1].imshow(stack[1,:,:], cmap='Greys')
ax[2].imshow(stack[2,:,:], cmap='Greys')
ax[3].imshow(stack[3,:,:], cmap='Greys')

In [ ]:
# メタデータの読み込み
# relionで使われるstar形式を利用
fn_star = "./data/course6_bgal_proj.star"

# まずメタデータから各投影像の配向情報だけを抜き出す
anglelist = []
with open(fn_star) as f:
    lines = [s.rstrip() for s in f.readlines()]
    # starファイルの一部を表示
    for i in range(29):
        print(lines[i])
    
    is_particles = False
    is_loop = False
    for line in lines:
        if (line == "data_particles"):
            is_particles = True
        if is_particles and line == "loop_":
            is_loop = True
            continue
        if not (is_particles and is_loop):
            continue
        if line[0:1] != "_" and len(line) != 0:
            meta = line.split()
            anglelist.append([float(meta[0]),float(meta[1]),float(meta[2])])

# 必須ではないが、ndarrayに変換しておくと便利な場合が多い
angles = np.array(anglelist)

In [ ]:
# メタデータを少しだけ確認
print("shape of angles: ", angles.shape)
print("metadata of particle 1:")
print(angles[0,:])

In [ ]:
# 三次元再構成計算

ndata = stack.shape[0]
boxsize = stack.shape[1]
padsize = int(boxsize * 2)
margin = int((padsize - boxsize) / 2)

# 必要な配列を初期化
rec3d = np.zeros((padsize,padsize,padsize), dtype=complex) # 三次元再構成される逆空間像
weight = np.zeros((padsize,padsize,padsize)) # 各ボクセルへ差し込まれた粒子像の枚数

print("backprojection:")
# 投影像ごとにループ処理
for i in range(ndata):
    # 各投影像の配向から回転行列を計算
    rot = angles[i,0]
    tilt = angles[i,1]
    psi = angles[i,2]
    A = np.zeros((3,3))
    projector.getRotationMatrix(A, rot=rot, tilt=tilt, psi=psi)

    # 投影像の逆空間像を計算
    padded = np.pad(stack[i,:,:], (margin,margin))
    rec2d = fft.fftshift(fft.fft2(fft.fftshift(padded)))

    # 三次元逆空間像への差し込み
    backprojector.backproject2Dto3D(rec2d, rec3d, weight, A)

    # 処理状況を逐次表示
    message  = str(i+1).format('>4') + " / " + str(ndata).format('>4') + " complete"
    print("\r"+message,end="")

print('\n')
print('reconstructing...')

# 投影像を差し込んで作った三次元逆空間像のリウェイト
rec3d_reweight = backprojector.reconstruct(rec3d, weight)

# 振幅による逆空間像のプロット
ampl = np.abs(rec3d_reweight)
second_min = np.unique(ampl)[1]
for i in range(rec3d.shape[0]):
    for j in range(rec3d.shape[1]):
        for k in range(rec3d.shape[2]):
            val = ampl[i,j,k]
            if val < second_min:
                ampl[i,j,k] = np.log(second_min)
            else:
                ampl[i,j,k] = np.log(ampl[i,j,k])

fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(12,4))

ax[0].imshow(ampl[90,:,:], cmap='viridis') # x-y
ax[1].imshow(ampl[:,90,:], cmap='viridis') # z-x
ax[2].imshow(ampl[:,:,90], cmap='viridis') # y-z

# mrcファイルへの書き込み
b = boxsize
m = margin

# 逆フーリエ変換で3Dマップへ変換
bgal3d_pad = fft.ifftshift(fft.ifftn(fft.ifftshift(rec3d_reweight),s=(padsize,padsize,padsize))).real

# mrc形式ではpython標準の倍精度(64ビット)実数が使えないので単精度(32ビット)へ変換する
bgal3d = np.zeros((boxsize,boxsize,boxsize),dtype=np.float32)
for i in range(boxsize):
    for j in range(boxsize):
        for k in range(boxsize):
            bgal3d[i,j,k] = bgal3d_pad[i+m,j+m,k+m]
fn_out = "./course06/bgal_bp.mrc"
mrc.write(fn_out, bgal3d, overwrite=True, voxel_size=2.548)

print("")
print("Wrote map: ", fn_out)

**実習課題**
 * 以下のデータを使ってプロジェクションマッチングと三次元再構成をやってみましょう
 * データについて、使われた配向はわかりますが粒子像ごとの配向は未知です。
 * projectionについては最後のセルを参考にしてください
 * 興味がある人は低分解能からスタートして繰り返しやってみてください(ただし計算は重いです)

In [ ]:
# 実習課題用データ
fn_vol = "data/course6_splc.mrc"
fn_stack = "data/course6_splc_proj.mrcs"
fn_angles = "data/course6_splc_angles.star"

stack = mrc.read(fn_stack)

fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(12,3))

ax[0].imshow(stack[0,:,:], cmap='Greys')
ax[1].imshow(stack[1,:,:], cmap='Greys')
ax[2].imshow(stack[2,:,:], cmap='Greys')
ax[3].imshow(stack[3,:,:], cmap='Greys')

In [ ]:
# projectionの例
fn_vol = "./data/course6_bgal.mrc"

vol = mrc.read(fn_vol)

print(vol.shape)

A = np.zeros((3,3))
projector.getRotationMatrix(A, rot=45, tilt=80.406, psi=346.5)

img = np.zeros((vol.shape[1],vol.shape[2]))

img = projector.project(vol,A)
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(8,4))

ax[0].imshow(img, cmap='Greys')
ax[1].imshow(stack[0,:,:], cmap='Greys')
